## Astrometry catalog (DEVELOPMENT, DOESN'T RUN....)

Process an association and derive an astrometric catalog to send to the `astrometry_db` database.

For fields that don't have a good existing astrometric reference (i.e., the pointings *not* in COSMOS, GOODS), derive a catalog from one JWST filter, e.g., F444W, and then align all other JWST filters to that one.

In [3]:
import os
os.environ['CRDS_PATH'] = '/tmp/crds_cache'
os.environ['CRDS_SERVER_URL'] = "https://jwst-crds.stsci.edu"

os.environ['iref'] = '/tmp/crds_cache/iref'
os.environ['jref'] = '/tmp/crds_cache/jref'

import glob

import warnings
warnings.filterwarnings('ignore')

import numpy as np
import matplotlib.pyplot as plt

from grizli import utils, prep
from grizli.aws import db, visit_processor

import jwst
print('jwst', jwst.__version__)

import grizli
print('grizli', grizli.__version__)


jwst 1.10.2
grizli 1.8.6


In [12]:
# Find PANORAMIC visits that haven't been processed

panoramic_assoc = db.SQL("""select assoc_name, max(status) as status, max(filter) as filter
from assoc_table
where status != 2 and filter in ('F444W-CLEAR')
and proposal_id in ('2514')
GROUP BY assoc_name
"""
) 

assoc_list = panoramic_assoc['assoc_name']

assoc_list = ['j100736p2109_indef-f444w_00023']

for assoc in assoc_list:
    print(assoc)
    
    if True:
        visit_processor.process_visit(assoc, sync=False, clean=False)


j100736p2109_indef-f444w_00023
# (2023-05-10 15:58:35.478)
# get_wcs_guess : 'j100736p2109_indef-f444w_00023'
j100736p2109_indef-f444w_00023/Prep/indef-02514-052-112.0-nrca5-f444w-clear.align_guess : 0.22 -0.08 0.0 1.0

Fetch params from s3://grizli-v2/HST/Pipeline/Input/j100736p2109_indef-f444w_00023.yaml

# Write parameters to j100736p2109_indef-f444w_00023.run.yaml



2023-05-10 15:58:38,324 - stpipe - INFO - jw02514052001_06201_00001_nrcalong_rate.fits exists, skip
2023-05-10 15:58:38,325 - stpipe - INFO - jw02514052001_06201_00002_nrcalong_rate.fits exists, skip


j100736p2109_indef-f444w_00023 : found 14 GAIA sources
j100736p2109_indef-f444w_00023.gaia.reg: x = ra, y=dec, ellipse=False
# (2023-05-10 15:58:38.298)

auto_script.fetch_files(**{'field_root': 'j100736p2109_indef-f444w_00023', 'HOME_PATH': '$PWD', 'paths': {'home': '/workspaces/panoramic-jwst/Notebooks', 'base': '/workspaces/panoramic-jwst/Notebooks/j100736p2109_indef-f444w_00023', 'raw': '/workspaces/panoramic-jwst/Notebooks/j100736p2109_indef-f444w_00023/RAW', 'prep': '/workspaces/panoramic-jwst/Notebooks/j100736p2109_indef-f444w_00023/Prep', 'persist': '/workspaces/panoramic-jwst/Notebooks/j100736p2109_indef-f444w_00023/Persistence', 'extract': '/workspaces/panoramic-jwst/Notebooks/j100736p2109_indef-f444w_00023/Extractions', 'thumbs': '/workspaces/panoramic-jwst/Notebooks/j100736p2109_indef-f444w_00023/Extractions'}, 'inst_products': {'ACS/WFC': ['FLC'], 'WFC3/IR': ['RAW'], 'WFC3/UVIS': ['FLC'], 'WFPC2/PC': ['C0M', 'C1M'], 'WFPC2/WFC': ['C0M', 'C1M']}, 'remove_bad': True, 'reproc

cp: cannot stat '*mask.reg': No such file or directory


Skip indef-02514-052-112.0-nrca5-f444w-clear
kill='preprocess'
Update exposure footprints in ./j100736p2109_indef-f444w_00023_visits.yaml


In [13]:
! cat j*/Prep/*wcs.log

# ext xshift yshift rot scale rms N
# radec: j100736p2109_f444w_00023_cat.radec
# indef-02514-052-112.0-nrca5-f444w-clear
    0        0.1823       -0.1096       -0.0138       1.00000         0.105  107


In [11]:
# Send visit
mag_lim = (17, 26)
flux_radius = (1,10)
sn = 20

max_count = 1000

HOME = os.getcwd()

for assoc in assoc_list:
    os.chdir(HOME)
    
    if os.path.exists(f'{assoc}/Prep'):
        os.chdir(f'{assoc}/Prep')
        files = glob.glob('*cat.fits')
        for file in files:
            tab = utils.read_catalog(file)
            for c in list(tab.colnames):
                tab.rename_column(c, c.lower())

            tab['mag'] = tab['mag_auto']
            if True:
                #clip = tab['mag'] < mag_lim[1]
                clip = tab['mag'] > mag_lim[0]
                clip &= tab['mask_aper_1'] == 0
                clip &= tab['flux_radius'] > flux_radius[0]
                clip &= tab['flux_radius'] < flux_radius[1]
                clip &= tab['flux_aper_1'] / tab['fluxerr_aper_1'] > sn
                
                tab = tab[clip]
                so = np.argsort(tab['mag'])
                tab = tab[so]
                
                if max_count > 0:
                    tab = tab[:max_count]
                    
            plt.scatter(tab['mag_auto'], tab['flux_radius'], alpha=0.2)
            plt.ylim(0, 10)
            plt.xlim(12,28)
            
            if len(files) == 1:
                src = assoc+'_cat'
            else:
                src = os.path.basename(file).split('.cat')[0]
            
            src = src.replace('indef-','')
            
            tab['src'] = src
            try:
                ok = np.isfinite(tab['mag'].filled(np.nan))
                tab = tab[ok]
            except:
                pass

            print(f'Add {len(tab)} objects to astrometry_referenc for {assoc}/{file} src={src}')
            
            prep.table_to_regions(tab, f'/tmp/{src}.reg')
            
            # Set this to True to actually send the table!
            if False:
                print(f'clean {src}')
                db.execute(f"delete from astrometry_reference where src='{src}'")
                print(f'upload {src}')
                db.send_to_database('astrometry_reference', tab['ra','dec','src','mag'], if_exists='append')

os.chdir(HOME)

Add 218 objects to astrometry_referenc for j100736p2109_indef-f444w_00023/indef-02514-052-112.0-nrca5-f444w-clear.cat.fits src=j100736p2109_f444w_00023_cat
/tmp/j100736p2109_f444w_00023_cat.reg: x = ra, y=dec, ellipse=False


<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=a56be2b0-05e9-4804-afc1-e2fa4997b5d9' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>